In [1]:
import re
import math
import traci
import pandas as pd
import random
from xml.dom import minidom

class Controller():

    #添加公交线路
    #route <- int
    def __init__(self,route):
        self.route=route
        self.viewport = pd.read_csv("csv_files/viewport.csv")[str(self.route)]
        self.focus_dict={0:0,1:47,2:48,3:263}
        self.lastBus=None
        self.perosonNum=0
        self.busNum=0
        self.busList={}

        #初始化公交线路
        edges=list(pd.read_csv("csv_files/route{}.csv".format(self.route))["route{}".format(self.route)])
        traci.route.add(str(self.route),edges)

        #获取站点列表
        doc=minidom.parse("xml_files/stop_{}.add.xml".format(self.route))
        self.stops=doc.getElementsByTagName("busStop")
        self.stopNum=len(self.stops)

        for stop in self.stops:
            self.busList[stop.getAttribute("id")]=(stop.getAttribute("lane"),stop.getAttribute("startPos"),stop.getAttribute("endPos"))




    #添加公交车，注意默认载客量50,默认最大速度10m/s
    def addBus(self):
        traci.vehicle.add("bus{}".format(self.busNum),str(self.route),line=str(self.route),typeID="bus",depart="now",departPos="0.0")
        self.lastBus="bus{}".format(self.stopNum)
        self.busNum+=1

    #添加乘客，这里仅设置其等待此刻已发车且还未到达的车辆，乘客等待的车辆实际上可能时刻发生变化
    def addPassenger(self,busStop):
        traci.person.add(busStop+"_{}".format(self.perosonNum),self.busList[busStop][0][:-2],random.uniform(float(self.busList[busStop][1]),float(self.busList[busStop][2])))
        #添加乘客乘坐线路以及下车地点
        if busStop[-2]=='_':
            stop=int(busStop[-1])+1
        else:
            stop=int(busStop[-2:])+1
        stopID="{}_{}".format(self.route,int(random.random()*(self.stopNum-stop))+stop)
        traci.person.appendDrivingStage(busStop+"_{}".format(self.perosonNum),toEdge=self.busList[busStop][0][:-2],lines=str(self.route),stopID=stopID)
        self.perosonNum+=1

    #聚焦某一条线路
    def focusRoute(self):
        traci.gui.setZoom("View #0",self.viewport[0])
        traci.gui.setOffset("View #0",self.viewport[1],self.viewport[2])
        traci.gui.setAngle("View #0",self.viewport[3])    
    
    #追踪最近发出的车辆
    def trackBus(self):
        traci.gui.track(self.lastBus)
    
    def highlightRoute(self):
        doc=minidom.parse("xml_files/highlight{}.poly.xml".format(self.route))
        route=doc.getElementsByTagName("poly")[0]
        colorstring=route.getAttribute("color")
        shapestring=route.getAttribute("shape")
        color= tuple(map(int,re.findall(r'\d+',colorstring)))
        shape=[tuple(map(float,item.split(","))) for item in shapestring.split(" ")]
        layer=route.getAttribute("layer")
        traci.polygon.add("route{}".format(self.route),shape=shape,color=color,layer=layer,lineWidth=20.0)
    
    def removeHighlight(self):
        traci.polygon.remove("route{}".format(self.route))

    def drawStopPoi(self,r=40):
        stopDict={47:37,48:28,263:28}
        stopNum=stopDict[self.route]
        pointsNum=10       #polygon点数
        r=r               #polygon圆形半径
        angle=(2*math.pi)/pointsNum
        for stop in self.stops:
            edge=stop.getAttribute("lane")[:-2]
            pos=float(stop.getAttribute("startPos"))+10.0
            shape=[]
            X,Y=traci.simulation.convert2D(edge,pos)
            for i in range(pointsNum+1):
                x=X+r*math.sin(i*angle)
                y=Y+r*math.cos(i*angle)
                shape.append((x,y))
            
            traci.polygon.add("stop"+stop.getAttribute("id"),lineWidth=100,shape=shape,color=(250,8,8),layer=102,fill=True)
        
    def removeStopPoi(self):
        stopDict={47:37,48:28,263:28}
        for i in range(stopDict[self.route]):
            traci.polygon.remove("stop{}_{}".format(self.route,i))
        






    #########################################统计信息#######################################
    # def 








    

sumoBinary="sumo-gui"
sumoConfig=[sumoBinary,'-c',"xml_files/guiyang.sumocfg"]
traci.start(sumoConfig)

line47_controller=Controller(route=47)
line48_controller=Controller(route=48)
#line47_controller.focusRoute()

#line47_controller.drawStopPoi()

#while traci.simulation.getMinExpectedNumber()>0:
#TEST    每60s发一次车，添加一名乘客，第100s切换线路焦点
step=0
stageChange=True
while step<9000:
    step+=1
    traci.simulation.step()
    if step==5:
        line47_controller.removeStopPoi()
        line47_controller.drawStopPoi(100)
    if stageChange:
        #line47_controller.highlightRoute()
        line47_controller.focusRoute()
        stageChange=False
    if step%60==0:
        line47_controller.addBus()
        line47_controller.addPassenger("47_1")
    
    if step==100:
        line47_controller.removeHighlight()
        line48_controller.highlightRoute()
        line48_controller.focusRoute()

        #highlight the route
        


TraCIException: Could not add polygon 'route47'